### Takes all current CDC probes and maps them against most recent sequences as they appear

This is a work in progress, the scripts here yrack the RT-PCR assays currently being developed around the world and can be used to determine if they all work for all the circulating strains, given by GISAID.


1. Collect Most recent CoronaVirus sequences and primers:

I posted this here: https://www.dropbox.com/sh/h7m6c6adrea1jxz/AACDd1WYE0gy2jN5xWk_wfq7a?dl=0
  
  It has all the rt-PCR primers I've gotten so far as well as the latest GIAID equences.  
  I'll keep this updated.

2. Map Seqs to Genome

3. Generate Labeled Databse: corona_<timestamp>_DB

4. Runs primers against corona_<timestamp>_DB using simple BLAST.
    
Eventually, I will also computes Thermo WillItHyb willItPCR  for partial matches.
    Like this study: https://graphgenomics.com/2020/02/20/pcr-predict-a-study-that-applies-machine-learning-techniques-to-the-real-worl/
    

In [1]:
#!pip install biopython
import sys, os, copy, pprint, glob
import datetime, json
#import pathlib
import matplotlib.pyplot as plt

from importlib import reload
import pathlib, importlib
import csv, fnmatch, random
import Bio.Entrez #used by accessions_to_gb
from ftplib import FTP
from io import StringIO
import pandas as pd
import numpy as np

sys.path.append('src/')
import BLAST.BlastRun as BlastRun
import utils.IO as IO
import 

IO.py


In [2]:
#Set a directory for work add sequences there:
ud="/tdata1/betacoronavirus/"

In [3]:
reload(IO)
tc=IO.TimeCode()
print(tc)

IO.py
20200402T0853


## Get Latest Corona Seqs:

### World Primer Set

In [5]:
#Thi sis generated using the spreadsheet: Corona_RT-PCRprimers.xlsx

primerLst=[
       ['G1|F|CDC|2019-nCoV_N1-F', 'GACCCCAAAATCAGCGAAAT'],
['G1|R|CDC|2019-nCoV_N1-R', 'TCTGGTTACTGCCAGTTGAATCTG'],
['G1|P|CDC|2019-nCoV_N1-P', 'ACCCCGCATTACGTTTGGTGGACC'],
['G2|F|CDC|2019-nCoV_N2-F', 'TTACAAACATTGGCCGCAAA'],
['G2|R|CDC|2019-nCoV_N2-R', 'GCGCGACATTCCGAAGAA'],
['G2|P|CDC|2019-nCoV_N2-P', 'ACAATTTGCCCCCAGCGCTTCAG'],
['G3|F|CDC|2019-nCoV_N3-F', 'GGGAGCCTTGAATACACCAAAA'],
['G3|R|CDC|2019-nCoV_N3-R', 'TGTAGCACGATTGCAGCATTG'],
['G3|P|CDC|2019-nCoV_N3-P', 'AYCACATTGGCACCCGCAATCCTG'],
['G4|F|CDC|RP-F', 'AGATTTGGACCTGCGAGCG'],
['G4|R|CDC|RP-R', 'GAGCGGCTGTCTCCACAAGT'],
['G4|P|CDC|RP-P', 'TTCTGACCTGAAGGCTCTGCGCG'],



['G5|P|CDC-2|IN-2', 'GGGTTGGGACTATCCTAAGTGTGA'],
['G6|P|CDC-2|IN-4', 'TAACACACAACICCATCATCA'],
['G5|F|CDC-2|Cor-p-F2', 'CTAACATGCTTAGGATAATGG'],
['G6|F|CDC-2|Cor-p-F3', 'GCCTCTCTTGTTCTTGCTCGC'],
['G5|R|CDC-2|Cor-p-R1', 'CAGGTAAGCGTAAAACTCATC'],
['G6|R|CDC-3|Cor-p-R2', 'CAGGTAAGCGTAAAACTCATC'],

['G7|F|China|ORF1ab_F', 'CCCTGTGGGTTTTACACTTAA '],
['G7|R|China|ORF1ab_R', 'ACGATTGTGCATCAGCTGA '],
['G7|P|China|ORF1ab_P', 'CCGTCTGCGGTATGTGGAAAGGTTATGG'],
['G8|F|China|N_F', 'GGGGAACTTCTCCTGCTAGAAT '],
['G8|R|China|N_R', 'CAGACATTTTGCTCTCAAGCTG '],
['G8|P|China|N_P', 'TTGCTGCTGCTTGACAGATT'],

['G10|F|Germany|RdRP_SARSr-F2', 'GTGARATGGTCATGTGTGGCGG'],
['G9|R|Germany|RdRP_SARSr-R1', 'CARATGTTAAASACACTATTAGCATA'],
['G10|P|Germany|RdRP_SARSr-P2', 'CAGGTGGAACCTCATCAGGAGATGC'],
['G9|P|Germany|RdRP_SARSr-P1', 'CCAGGTGGWACRTCATCMGGTGATGC'],
['G9|F|Germany|E_Sarbeco_F1', 'ACAGGTACGTTAATAGTTAATAGCGT'],
['G10|R|Germany|E_Sarbeco_R2', 'ATATTGCAGCAGTACGCACACA'],
['G9|P|Germany|E_Sarbeco_P1', 'ACACTAGCCATCCTTACTGCGCTTCG'],

['G11|F|HongKong|HKU-ORF1b-nsp14F', 'TGGGGYTTTACRGGTAACCT'],
['G11|R|HongKong|HKU-ORF1b-nsp14R', 'AACRCGCTTAACAAAGCACTC'],
['G11|P|HongKong|HKU-ORF1b-nsp141P', 'TAGTTGTGATGCWATCATGACTAG'],
['G12|F|HongKong|HKU-NF', 'TAATCAGACAAGGAACTGATTA'],
['G12|R|HongKong|HKU-NR', 'CGAAGGTGTGACTTCCATG'],
['G12|P|HongKong|HKU-NP', 'GCAAATTGTGCAATTTGCGG'],

['G13|F|Thailand|WH-NIC_N-F', 'CGTTTGGTGGACCCTCAGAT'],
['G13|R|Thailand|WH-NIC_N-R', 'CCCCACTGCGTTCTCCATT'],
['G13|P|Thailand|WH-NIC_N-P', 'CAACTGGCAGTAACCA'],

['G14|F|Japan|NIID_WH-1_F501_F', 'TTCGGATGCTCGAACTGCACC'],
['G14|R|Japan|NIID_WH-1_R913_R', 'CTTTACCAGCACGTGCTAGAAGG'],
['G14|F|Japan|NIID_WH-1_F509_F', 'CTCGAACTGCACCTCATGG'],
['G15|R|Japan|NIID_WH-1_R854_R', 'CAGAAGTTGTTATCGACATAGC'],
['G15|F|Japan|NIID_WH-1_Seq_F519_F', 'ACCTCATGGTCATGTTATGG'],
['G15|R|Japan|NIID_WH-1_Seq_R840_R', 'GACATAGCGAGTGTATGCC'],
['G16|F|Japan|WuhanCoV-spk1-f_F', 'TTGGCAAAATTCAAGACTCACTTT'],
['G16|R|Japan|WuhanCoV-spk2-r_R', 'TGTGGTTCATAAAAATTCCTTTGTG'],
['G17|F|Japan|NIID_WH-1_F24381_F', 'TCAAGACTCACTTTCTTCCAC'],
['G17|R|Japan|NIID_WH-1_R24873_R', 'ATTTGAAACAAAGACACCTTCAC'],
['G18|F|Japan|NIID_WH-1_Seq_F24383_F', 'AAGACTCACTTTCTTCCACAG'],
['G18|R|Japan|NIID_WH-1_Seq_R24865_R', 'CAAAGACACCTTCACGAGG'],
['G19|F|Japan|NIID_2019-nCOV_N_F2_F', 'AAATTTTGGGGACCAGGAAC'],
['G19|R|Japan|NIID_2019-nCOV_N_R2_R', 'TGGCAGCTGTGTAGGTCAAC'],
['G19|P|Japan|NIID_2019-nCOV_N_P2_P', 'ATGTCGCGCATTGGCATGGA'],

['G20|F|France|nCoV_IP2-12669Fw_F', 'ATGAGCTTAGTCCTGTTG'],
['G20|R|France|nCoV_IP2-12759Rv_R', 'CTCCCTTTGTTGTGTTGT'],
['G20|P|France|nCoV_IP2-12696bProbe(+)', 'AGATGTCTTGTGCTGCCGGTA'],
['G21|F|France|nCoV_IP4-14059Fw', 'GGTAACTGGTATGATTTCG'],
['G21|R|France|nCoV_IP4-14146Rv', 'CTGGTCAAGGTTAATATAGG'],
['G21|P|France|nCoV_IP4-14084Probe(+)', 'TCATACAAACCACGCCAGG'],
['G22|F|France|E_Sarbeco_F1', 'ACAGGTACGTTAATAGTTAATAGCGT'],
['G22|R|France|E_Sarbeco_R2', 'ATATTGCAGCAGTACGCACACA'],
['G22|P|France|E_Sarbeco_P1', 'ACACTAGCCATCCTTACTGCGCTTCG'],

['G25|F|eurosurveillance|RdRp_SARSr-F', 'GTGARATGGTCATGTGTGGCGG'],
['G25|P1|eurosurveillance|RdRp_SARSr-P2', 'CAGGTGGAACCTCATCAGGAGATGC'],
['G25|P|eurosurveillance|RdRP_SARSr-P1', 'CCAGGTGGWACRTCATCMGGTGATGC'],
['G25|R|eurosurveillance|RdRp_SARSr-R', 'CARATGTTAAASACACTATTAGCATA'],
['G26|F|eurosurveillance|E_Sarbeco_F', 'ACAGGTACGTTAATAGTTAATAGCGT'],
['G26|P|eurosurveillance|E_Sarbeco_P1', 'ACACTAGCCATCCTTACTGCGCTTCG'],
['G26|R|eurosurveillance|E_Sarbeco_R', 'ATATTGCAGCAGTACGCACACA'],
['G27|F|eurosurveillance|N_Sarbeco_F', 'CACATTGGCACCCGCAATC'],
['G27|P|eurosurveillance|N_Sarbeco_P', 'ACTTCCTCAAGGAACAACATTGCCA'],
['G27|R|eurosurveillance|N_Sarbeco_R', 'GAGGAACGAGAAGAGGCTTG'],
]

## 1. Collate Sequences/FDB/RunBlast:

In [21]:
reload(BlastRun)
bg_fileLst=[
                'gisaid_cov2020_sequences.fasta',
    
    #uncommenting these lines does run against all coronviruses
#             'alphacorona_sequence_genomes.fasta', 
#             'deltacorona_sequence_genomes.fasta', 
#             'gammacorona_sequence_genomes.fasta', 
#             'betacorona_sequence_genomes.txt',
           ]
query_path='corona_PCRPrimers.fa'
bg_pth=ud+'gisaid_cov2020.fa'

fastaLst=[];subj_seq_dict={}
fp=open(bg_pth, 'w')
for bg_fn in bg_fileLst:
    group_name=bg_fn.split('_')[0]
    if group_name=='gisaid':
        group_name='covid19'
    c_type=bg_fn.split('_')[0]
    for faObj in IO.inFASTA_yield(ud+bg_fn):
        fastaLst.append([c_type, faObj[0]])
        label, seq=faObj[:2]
        if '[' in label:
            label=label.replace('[', '').replace(']', '')
            lablst=label.split(' ')
            label='|'.join(lablst)
            
        label=label.split(' ')[0]#Blast cuts your label off after the first space
        label="%s|%s|%i"%(group_name, label, len(seq))
        fp.write(">%s\n%s\n"%(label, seq))
        subj_seq_dict[label]=seq
fp.close()

df=pd.DataFrame(fastaLst, columns=['c_type', 'fa_label'])
#df.to_csv(ud+'fasta_labels.csv')

fdb_path=ud+'corona_bg'
subj_FDB = BlastRun.FormatDB(bg_pth, fdb_path, input_type='fasta', dbtype = 'nucl', title = "")
subj_FDB.RunFromFDB(parse_seqids=False)

inFASTA_yield(/tdata1/betacoronavirus/gisaid_cov2020_sequences.fasta)  num_seqs: 3228
cmdstr: cd /tdata1/betacoronavirus;makeblastdb  -in /tdata1/betacoronavirus/gisaid_cov2020.fa -out /tdata1/betacoronavirus/corona_bg -input_type fasta -dbtype nucl -parse_seqids;
cmdstr: blastdbcmd -db /tdata1/betacoronavirus/corona_bg -entry all -outfmt "%t %l"|tee /tdata1/betacoronavirus/SLST.TXT
ListBlastDB: returning 3228 recs
cmdstr: blastdbcmd -db /tdata1/betacoronavirus/corona_bg -entry all -outfmt "%t %l"|tee /tdata1/betacoronavirus/SLST.TXT
ListBlastDB: returning 3228 recs


'/tdata1/betacoronavirus/corona_bg'

## 2. Map Seqs to Genome

In [22]:
reload(BlastRun)
import json

class primer_hit:
    def __init__(self, hit, query_fa):
        self.query_name, self.query_seq = query_fa[:2]
        self.query_len=len(self.query_seq)
        self.subj_label=hit['description'][0]['title']
        self.subj_acc=hit['description'][0]['accession']
        self.num_hsp=len(hit['hsps'])
        self.hit_id=hit['num']
        hsp=hit['hsps'][0]
        self.score=hsp['score']
        self.identity=hsp['identity']
        self.similarity_score=self.identity/float(self.query_len)
        
        self.query_start=int(hsp['query_from'])
        self.query_to=int(hsp['query_to'])
        self.query_strand=hsp['query_strand']
        
        self.subj_start=int(hsp['hit_from'])
        self.subj_to=int(hsp['hit_to'])
        self.subj_strand=hsp['hit_strand']
        self.same_strand=self.query_strand==self.subj_strand
        self.subj_seq=''
    def set_subj_seq(self, subj_dict, seq_buffer=20):
        s=min(0, min(self.subj_start, self.subj_to)-seq_buffer)
        e=max(self.subj_start, self.subj_to)+seq_buffer
        self.subj_seq=subj_dict[self.subj_label][s:e]
    
    



class PCRRunner:
    def __init__(self, ud, primerLst, blast_fdb, subj_seq_dict, group='test'):
        self.ud=ud
        self.group=group
        self.primer_dict=dict([label,seq] for label,seq in primerLst)
        self.bi_path=self.ud+'corona_PCRPrimers_%s.fa'%self.group
        self.make_primer_BI()
        self.blast_fdb=blast_fdb
        self.subj_seq_dict=subj_seq_dict
        self.subj_template_dict={}
        self.blast_params={
                  'dust':"no",
                  'outfmt':15,
                  'num_threads':15,
                  'word_size':7,
                  'max_target_seqs': 100000
                  }
        
        self.rec=dict((k.split('|')[1],k) for k in self.primer_dict.keys())
        self.run_blast()
    def make_primer_BI(self):
        ud="/tdata1/betacoronavirus/"
        fp=open(self.bi_path, 'w')
        fp.write('\n'.join(['>%s\n%s'%(label, seq) for label, seq in self.primer_dict.items()]))
        fp.close()
    def run_blast(self):
        print("run_blast")
        self.bo_path=ud+'bo_primer_corona.txt'
        BR=BlastRun.BlastRunObj(self.bi_path, self.bo_path, self.blast_fdb, blast_params=self.blast_params)
        print(BR.cmd)
        self.bo_path=BR.RunBlast()
    def parse_blast(self, seq_buffer=10):
        bo_json=json.loads(open(self.bo_path).read())
        self.primer_hit_dict={}
        for bb in bo_json["BlastOutput2"]:
            report=bb["report"]
            query_label=report['results']["search"]["query_title"]
            query_seq=self.primer_dict[query_label]
            query_fa=[query_label, query_seq]
            if not query_label in self.primer_hit_dict:
                self.primer_hit_dict[query_label]={}
            for hit in report['results']["search"]["hits"]:
                ph=primer_hit(hit, query_fa)
                if not(ph.subj_label in self.primer_hit_dict[query_label]):
                    self.primer_hit_dict[query_label][ph.subj_label]=[]
                self.primer_hit_dict[query_label][ph.subj_label].append(ph)
        #set subj_seq
        oLst=[]
        for query_label in self.primer_hit_dict.keys():
            for subj_label in self.primer_hit_dict[query_label].keys():
                phLst=self.primer_hit_dict[query_label][subj_label]
                subj_se=[]
                for ph in phLst:
                    subj_se.append(ph.subj_start)
                    subj_se.append(ph.subj_to)
                s_start=min(subj_se)
                s_end=max(subj_se)
                s=max(0, s_start-seq_buffer)
                e=s_end+seq_buffer
                subj_seq=self.subj_seq_dict[subj_label][s:e]
                rec=[subj_label, s_start, s_end, subj_seq]
                if not(subj_label in self.rec):self.rec[subj_label]=0
                for ph in phLst:
                    ph.subj_seq=subj_seq
                    self.rec[subj_label]+=ph.similarity_score
    def compute_pcr_wip(self):
        if self.same_strand:
            pcr_wip=SWT.TM.WillItPCR(self.subj_seq, f_primer, r_primer)
        else:
            s=amplicon_hit.send-buffer
            e=amplicon_hit.sstart+buffer
            hit_seq=seq_pos_dict[amplicon_hit.subject].getSeq()[s:e]
            as_hit_seq=MolBio.Antisense(hit_seq)
            sas='anti'
            pcr_wip=SWT.TM.WillItPCR(as_hit_seq, MolBio.Antisense(r_primer), MolBio.Antisense(f_primer))
        pcr_wip['amplicon_name']=q_label
        pcr_wip['amplicon']=amplicon
        pcr_wip['amplicon_region']=hit_seq
        pcr_wip['amplicon_region_len']=len(hit_seq)
        pcr_wip['amplicon_pos']="subj:%s|%i:%i|len:%i"%(amplicon_hit.subject, s,e,len(hit_seq))
        pcr_wip['f_primer']=f_primer
        pcr_wip['r_primer']=r_primer
        pcr_wip['sas']=sas
        pcr_wip['amplicon_hit']='|'.join(amplicon_hit.hitLst)

In [23]:
oLst=[]
for plst in IO.groupby(primerLst, key=lambda x:x[0].split('|')[0]):
    P=PCRRunner(ud, plst, fdb_path, subj_seq_dict, group='test')
    P.parse_blast(seq_buffer=10)
    oLst.append(P)
    
    
df=pd.DataFrame([P.rec for P in oLst])
df.to_csv(ud+'test_pcr_%s.csv'%tc)

run_blast
blastn  -query /tdata1/betacoronavirus/corona_PCRPrimers_test.fa -out /tdata1/betacoronavirus/bo_primer_corona.txt -db /tdata1/betacoronavirus/corona_bg -dust no -outfmt 15 -num_threads 15 -word_size 7 -max_target_seqs 100000;
run_blast
blastn  -query /tdata1/betacoronavirus/corona_PCRPrimers_test.fa -out /tdata1/betacoronavirus/bo_primer_corona.txt -db /tdata1/betacoronavirus/corona_bg -dust no -outfmt 15 -num_threads 15 -word_size 7 -max_target_seqs 100000;
run_blast
blastn  -query /tdata1/betacoronavirus/corona_PCRPrimers_test.fa -out /tdata1/betacoronavirus/bo_primer_corona.txt -db /tdata1/betacoronavirus/corona_bg -dust no -outfmt 15 -num_threads 15 -word_size 7 -max_target_seqs 100000;
run_blast
blastn  -query /tdata1/betacoronavirus/corona_PCRPrimers_test.fa -out /tdata1/betacoronavirus/bo_primer_corona.txt -db /tdata1/betacoronavirus/corona_bg -dust no -outfmt 15 -num_threads 15 -word_size 7 -max_target_seqs 100000;
run_blast
blastn  -query /tdata1/betacoronavirus/cor

In [ ]:
df.to_csv(ud+'test_pcr_%s.csv'%tc)

# NOT Implemented Yet

## 3. Runs primers against corona_timestamp_DB and computes Thermo WillItHyb willItPCR 

For imperfect primers: compute willItHyb

In [25]:
import Thermo.MolBio as MolBio
import Thermo.SWThermo as SWT
reload(GT)
reload(SWT)

def AlignHit(h, seq_pos_dict, aquery_dict, buffer=10):
    """
    This builds a pretty alignment
    """
    #print('aquery_dict[h.query]', aquery_dict[h.query])
    q_label, amplicon, f_primer, r_primer = aquery_dict[h.query]
    sas=''
    if h.sense:
        hit_seq=seq_pos_dict[h.subject].getSeq()[max(0,(h.sstart-buffer)):(h.send+buffer)]
    else:
        hit_seq=MolBio.Antisense(seq_pos_dict[h.subject].getSeq()[max(0,(h.send-buffer)):(h.sstart+buffer)])
        sas='anti'
    return '\"%s\"'%('\r'.join(["%s_%s-sense"%(q_label, sas),hit_seq, f_primer, r_primer]))

def ThermoPCRHit(amplicon_hit, seq_pos_dict, aquery_dict, buffer=10):
    """For amplicon_hit it retrieves sequence Plus buffer from subj DataBase Then it computes ThermoPCRHit
        amplicon_hit: GT.hit
        seq_pos_dict: from IO.SetUp_SequenceRetrievalSystem(glob_str, pool_sz=15)
        aquery_dict: stores (q_label, amplicon, f_primer, r_primer, full_seq) by probe query name
        buffer: how much extra sequence to retrieve from database
        """
    q_label, amplicon, f_primer, r_primer = aquery_dict[amplicon_hit.query]
    sas=''
    if amplicon_hit.sense:
        s=amplicon_hit.sstart-buffer
        e=amplicon_hit.send+buffer
        hit_seq=seq_pos_dict[amplicon_hit.subject].getSeq()[s:e]
        pcr_wip=SWT.TM.WillItPCR(hit_seq, f_primer, r_primer)
    else:
        s=amplicon_hit.send-buffer
        e=amplicon_hit.sstart+buffer
        hit_seq=seq_pos_dict[amplicon_hit.subject].getSeq()[s:e]
        as_hit_seq=MolBio.Antisense(hit_seq)
        sas='anti'
        pcr_wip=SWT.TM.WillItPCR(as_hit_seq, MolBio.Antisense(r_primer), MolBio.Antisense(f_primer))
    pcr_wip['amplicon_name']=q_label
    pcr_wip['amplicon']=amplicon
    pcr_wip['amplicon_region']=hit_seq
    pcr_wip['amplicon_region_len']=len(hit_seq)
    pcr_wip['amplicon_pos']="subj:%s|%i:%i|len:%i"%(amplicon_hit.subject, s,e,len(hit_seq))
    pcr_wip['f_primer']=f_primer
    pcr_wip['r_primer']=r_primer
    pcr_wip['sas']=sas
    pcr_wip['amplicon_hit']='|'.join(amplicon_hit.hitLst)
    return pcr_wip


def Run_ThermoPCRHit(query_faLst, subj_Lst, seq_pos_dict, bo_path, out_path, pcr_predict_model=None, SIM_SIM_LIMIT=None):
    """
    Takes PCR reactions and predicts hits to databases of genomes
    query_faLst:     'label', 'amplicon', 'pcr_for', 'pcr_rev'
    subj_Lst:         List of subj genomes from [x[0] for x in list(subj_FDB.ListBlastDB())]
    seq_pos_dict:     from IO.SetUp_SequenceRetrievalSystem(glob_str, pool_sz=15)
    bo_path:          Blast_output path
    out_path:         out_path
    SIM_SIM_LIMIT:    aux similarity threshold
    """
    _query_dict={}
    for p in query_faLst:
        _query_dict[p[0]]=p
    fasta_sz_dict={}
    for fa in query_faLst:
        fasta_sz_dict[fa[0]] = len(fa[1])
    ll = ['label']
    sll = ['align_label']
    oLst = [[subj_label] for subj_label in subj_Lst]
    sLst = [['align_%s'%subj_label] for subj_label in subj_Lst]
    thermoLst=[]
    N=0
    for hitLst in GT.StreamParseBlastQueryGroupBy(bo_path, hit_similarity_lambda = lambda x:x.percid>50.0, Verbosity=0.0):
        if not(hitLst):continue
        query = hitLst[0].query
        map_gene=query#.split('|')[0]
        print('query', query, 'num_hits: %i'%len(hitLst))
        q_len = fasta_sz_dict[query]
        ll.append("%s [len=%i]"%(query, q_len))
        sll.append("%s align"%(query))
        d={}
        for hit in hitLst:
            seq_sim = hit.SeqSimilarity(q_len)
            if SIM_SIM_LIMIT:
                if seq_sim<SIM_SIM_LIMIT:
                    print('SIM_SIM_LIMIT: skipping', hit.query, hit.subject, seq_sim)
                    continue
            hit.seq_sim = seq_sim
            esc = hit.subject
            if not(esc in d):
                d[esc]=hit
            if hit.seq_sim>d[esc].seq_sim:
                d[esc]=hit
        for i in range(len(oLst)):
            esc=oLst[i][0]
            if esc in d:
                oLst[i].append(d[esc].seq_sim)
            else:
                oLst[i].append(0)
        for i in range(len(sLst)):
            esc=oLst[i][0]
            if esc in d:
                #sLst[i].append(AlignHit(d[esc], seq_pos_dict, _query_dict))
                swip = ThermoPCRHit(d[esc], seq_pos_dict, _query_dict, buffer=60)
                swip['query_primerset']=query
                swip['subj_genome']=esc
                swip['blast_sim']=d[esc].seq_sim#BLAST Similarity
                thermoLst.append(swip)
                if pcr_predict_model:
                    sLst[i].append(PCR_model_Predict(swip, pcr_predict_model))
                else:
                    try:
                        sLst[i].append(min(swip['for_align_score'], swip['rev_align_score']))
                    except:
                        sLst[i].append('')
            else:
                sLst[i].append('n/a')
        N+=1
        #if N>0:break
    for i in range(len(oLst)):
        oLst[i].extend(sLst[i])
        oLst[i].append('_')
    LL=ll+sll+['sep']
    thermoLst_df=pd.DataFrame(thermoLst)
    thermoLst_df.to_csv(out_path+'_thermo.csv')
    df = pd.DataFrame.from_records(oLst, columns=LL)
    df.to_csv(out_path)
    return df, thermoLst_df

ModuleNotFoundError: No module named 'MolBio'

In [ ]:
import  joblib
import Probe.MolBio as MB
reload(MB)
reload(SWT)

pcr_predict_model=    joblib.load('/home/ubuntu/git/Genotyper/src/Probe/PCRPredictModel_depth6.joblib')
alt_pcr_predict_model=joblib.load('/home/ubuntu/git/Genotyper/src/Probe/PCRPredictModel_deltaTm_depth6.joblib')
def Compute_pcr_wip(amplicon, for_primer, rev_primer, align_sum_thresh=0.5):
    #Tests to make sure your anti setting is correct
    pcr_wip=SWT.TM.WillItPCR(amplicon, for_primer, rev_primer)
    pcr_wip['flipped_align_delta']=''
    align_sum=pcr_wip['for_align_score']+pcr_wip['rev_align_score']
    if align_sum<align_sum_thresh:
        pcr_wip=SWT.TM.WillItPCR(MB.Antisense(amplicon), for_primer, rev_primer)
        flipped_align_sum=pcr_wip['for_align_score']+pcr_wip['rev_align_score']
        pcr_wip['flipped_align_delta']=flipped_align_sum-align_sum
    return pcr_wip

def PCR_model_Predict(pcr_wip, pcr_predict_model):
    if not('for_Tm' in pcr_wip) or not('rev_Tm' in pcr_wip):
        return 0.0
    for_rev_delta=pcr_wip['for_Tm']-pcr_wip['rev_Tm']
    if for_rev_delta>0:
        fr='rev'
    else:
        fr='for'
    #attributes=['Tm', 'align_score', 'mis_matches', 'three_prime_mm', 'three_prime_overhang']
    attributes=['deltaTm', 'align_score','mis_matches', 'three_prime_mm', 'three_prime_overhang']
    d={}
    for k in attributes:
        d[k]=pcr_wip['%s_%s'%(fr, k)]
    d['three_prime_overhang']= 0 if d['three_prime_overhang']<0 else d['three_prime_overhang']
    d['three_prime_mm']= 0 if not(d['three_prime_mm']) else d['three_prime_mm']
    X=np.array([d[k] for k in attributes]).reshape(1, -1)
    d['predict']=pcr_predict_model.predict_proba(X)
    return d['predict'][0][1]

    
    
def PCRPredict(for_primer, rev_primer, amplicon, sas, 
               pcr_predict_model, 
               attributes=['align_score','mis_matches', 'three_prime_mm', 'three_prime_overhang'], 
               ReturnPassProb=True):
    if sas=='anti':
        amplicon=MB.Antisense(amplicon)
    pcr_wip=Compute_pcr_wip(amplicon, for_primer, rev_primer, align_sum_thresh=0.5)
    if not('for_Tm' in pcr_wip) or not('rev_Tm' in pcr_wip):
        if ReturnPassProb:
            return 0.0
        else:
            return [0,0,0,0] + [0.0]
    for_rev_delta=pcr_wip['for_Tm']-pcr_wip['rev_Tm']
    if for_rev_delta>0:
        fr='rev'
    else:
        fr='for'
    d={}
    for k in attributes:
        d[k]=pcr_wip['%s_%s'%(fr, k)]
    d['three_prime_overhang']= 0 if d['three_prime_overhang']<0 else d['three_prime_overhang']
    d['three_prime_mm']= 0 if not(d['three_prime_mm']) else d['three_prime_mm']
    X=np.array([d[k] for k in attributes]).reshape(1, -1)
    d['predict']=pcr_predict_model.predict_proba(X)
    #d['predict']=clf.predict_proba(X)
    d['pass_prob']=d['predict'][0][1]
    #print('X', X)#, d['predict'])
    if ReturnPassProb:
        return d['pass_prob']
    if pcr_wip['flipped_align_delta']: print('flipped_align_delta', pcr_wip['flipped_align_delta'])
    return [d[k] for k in attributes] + [d['pass_prob']]
    


p=PCRPredict(for_primer, rev_primer, amplicon, 'anti', 
                           pcr_predict_model,
                           attributes=['align_score','mis_matches', 'three_prime_mm', 'three_prime_overhang'],
                           ReturnPassProb=False)
alt_p=PCRPredict(for_primer, rev_primer, amplicon, 'anti', 
                           alt_pcr_predict_model,
                           attributes=['deltaTm', 'align_score','mis_matches', 'three_prime_mm', 'three_prime_overhang'],
                           ReturnPassProb=False)



print(p, alt_p)

In [ ]:
bo_path="/tdata1/enterobase_20190205/BO_probedb.fa"
out_path = ud + 'CH_genome_map_align_model.csv' 

subj_name_Lst = [x[0] for x in list(subj_FDB.ListBlastDB())]

pcr_predict_model=joblib.load('/home/ubuntu/git/Genotyper/src/Probe/PCRPredictModel.joblib')
p_df, thermoLst_df=Run_ThermoPCRHit(faLst, subj_name_Lst, seqDict, bo_path, out_path, pcr_predict_model=pcr_predict_model)

## Import Alignment

In [ ]:
aln_path=ud+'gis_cov2020align.aln'


## JSON Schema Discovery for JSON Blast output

In [4]:
#json_schema of BLAST output format: 15 (JSON combined)

class jnode:
    def __init__(self, key, obj, parent='', tab=''):
        self.key=key
        self.obj=obj
        self.count=0
        self.cardinality=0
        self.type=str(type(obj))
        self.leaf=False
        if not(('dict' in self.type)or('list' in self.type)):
            self.leaf=True
        self.tab=tab
        self.dict_children=[]
        self.list_children=[]
        self.compute_path(parent)
        self.deep()
    def compute_path(self, parent):
        parent_path=''
        if parent:
            parent_path=parent.path
            if 'list' in parent.type:
                self.path=parent_path+"[%s]"%self.key
            else:
                self.path=parent_path+"['%s']"%self.key
        else:
            self.path=parent_path+"['%s']"%self.key
    def deep(self):
        #Fills out node representation
        if 'dict' in self.type:
            for key in self.obj.keys():
                self.dict_children.append(jnode(key, self.obj[key], parent=self, tab=self.tab+'\t'))
        if 'list' in self.type:
            for idx, obj in enumerate(self.obj):
                self.list_children.append(jnode('%i'%idx, obj, parent=self, tab=self.tab+'['))
        self.cardinality=len(self.dict_children)+len(self.list_children)
    def print(self, string, fp=None):
        if fp:
            fp.write("%s\t%s\n"%(self.path, string))
        else:
            print(string)
    def display(self, fp=None):
        if self.leaf:
            self.print('%s%s (%s) Leaf:%s'%(self.tab, str(self.key), self.type, str(self.obj)), fp=fp)
            return
        self.print('%s%s (%s) [n=%i]'%(self.tab, self.key, self.type, self.cardinality), fp=fp)
        for n in self.dict_children:
            n.display(fp=fp)
        for n in self.list_children:
            n.display(fp=fp)
    def compact(self):
        pass
    
                                  

J=jnode('root', bo_json)
fp=open(ud+'schema.txt', 'w')
J.display(fp=fp)
fp.close()
                                  
#bo_json

NameError: name 'bo_json' is not defined

## Deprecation Line

In [ ]:
class BlastResult:
    def __init__(self, hits, query_fa, subj_seq_dict):
        self.query_fa=query_fa
        self.query_label, self.query_seq=self.query_fa[:2]
        self.hitLst=[]
        self.subj_dict=subj_seq_dict
        self.alignmentLst=[]
        self.perfect_hits=0
        self.parse_hits(hits)
    def parse_hits(self, hits):
        for hit in hits:
            jHit=primer_hit(hit, self.query_fa)
            jHit.set_subj_seq(self.subj_dict, seq_buffer=20)
            self.hitLst.append(jHit)
        self.count_perfect_hits()
    def count_perfect_hits(self):
        self.perfect_hits=0
        for h in self.hitLst:
            if (1.0-h.similarity_score)<0.01:
                self.perfect_hits+=1
    def display(self):
        print(self.query_label, len(self.hitLst), self.perfect_hits)

        
            
def ParseJSONBlastOutput(bo_path, primer_dict, subj_seq_dict):
    bo_json=json.loads(open(bo_path).read())
    blst=[]
    for bb in bo_json["BlastOutput2"]:
        report=bb["report"]
        query_label=report['results']["search"]["query_title"]
        query_seq=primer_dict[query_label]
        query_fa=[query_label, query_seq]
        hits=report['results']["search"]["hits"]
        blst.append(BlastResult(hits, query_fa, subj_seq_dict))
    return blst